In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import lightgbm as lgb

In [3]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
	sys.path.insert(0, src_path)

from data_loader import load_data , load_clean_data
from features import add_time_features, create_target

In [5]:
df = load_clean_data("../data/processed/data.csv")

In [6]:
df.head()

,InvoiceDate,Country,StockCode,Year,Month,Day,Hour,min,sec,is_weekend,day_of_week,hour_sin,hour_cos,quarter,stockcode_freq,country_freq,target
0,2009-12-01 07:45:00,United Kingdom,0.466434,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,131,981330,1
1,2009-12-01 07:45:00,United Kingdom,0.551061,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,70,981330,1
2,2009-12-01 07:45:00,United Kingdom,0.547694,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,90,981330,1
3,2009-12-01 07:45:00,United Kingdom,0.385078,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,110,981330,1
4,2009-12-01 07:45:00,United Kingdom,0.210763,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,517,981330,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 17 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   InvoiceDate     1067371 non-null  object 
 1   Country         1067371 non-null  object 
 2   StockCode       1067371 non-null  float64
 3   Year            1067371 non-null  int64  
 4   Month           1067371 non-null  int64  
 5   Day             1067371 non-null  int64  
 6   Hour            1067371 non-null  int64  
 7   min             1067371 non-null  int64  
 8   sec             1067371 non-null  int64  
 9   is_weekend      1067371 non-null  int64  
 10  day_of_week     1067371 non-null  int64  
 11  hour_sin        1067371 non-null  float64
 12  hour_cos        1067371 non-null  float64
 13  quarter         1067371 non-null  int64  
 14  stockcode_freq  1067371 non-null  int64  
 15  country_freq    1067371 non-null  int64  
 16  target          1067371 non-null  in

In [8]:
X = df.drop(columns="target")
y = df["target"]

In [9]:
train_mask = df["InvoiceDate"] <= "2010-06-30"
val_mask   = (df["InvoiceDate"] > "2010-06-30") & (df["InvoiceDate"] <= "2010-09-30")
test_mask  = df["InvoiceDate"] > "2010-09-30"


In [10]:
X = X.drop('InvoiceDate', axis=1)
X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask], y[val_mask]
X_test,  y_test  = X[test_mask], y[test_mask]

In [11]:
import optuna
import sklearn.metrics

c:\Users\anasm\miniconda3\envs\torch-ds\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
CATEGORICAL_FEATURES = [
    'Country'
]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 17 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   InvoiceDate     1067371 non-null  object 
 1   Country         1067371 non-null  object 
 2   StockCode       1067371 non-null  float64
 3   Year            1067371 non-null  int64  
 4   Month           1067371 non-null  int64  
 5   Day             1067371 non-null  int64  
 6   Hour            1067371 non-null  int64  
 7   min             1067371 non-null  int64  
 8   sec             1067371 non-null  int64  
 9   is_weekend      1067371 non-null  int64  
 10  day_of_week     1067371 non-null  int64  
 11  hour_sin        1067371 non-null  float64
 12  hour_cos        1067371 non-null  float64
 13  quarter         1067371 non-null  int64  
 14  stockcode_freq  1067371 non-null  int64  
 15  country_freq    1067371 non-null  int64  
 16  target          1067371 non-null  in

In [14]:
# List of columns identified in the error
# Convert to category type
for col in CATEGORICAL_FEATURES:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

C:\Users\anasm\AppData\Local\Temp\ipykernel_27060\4100065008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype('category')
C:\Users\anasm\AppData\Local\Temp\ipykernel_27060\4100065008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[col] = X_val[col].astype('category')


In [15]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108572 entries, 255103 to 363674
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Country         108572 non-null  category
 1   StockCode       108572 non-null  float64 
 2   Year            108572 non-null  int64   
 3   Month           108572 non-null  int64   
 4   Day             108572 non-null  int64   
 5   Hour            108572 non-null  int64   
 6   min             108572 non-null  int64   
 7   sec             108572 non-null  int64   
 8   is_weekend      108572 non-null  int64   
 9   day_of_week     108572 non-null  int64   
 10  hour_sin        108572 non-null  float64 
 11  hour_cos        108572 non-null  float64 
 12  quarter         108572 non-null  int64   
 13  stockcode_freq  108572 non-null  int64   
 14  country_freq    108572 non-null  int64   
dtypes: category(1), float64(3), int64(11)
memory usage: 12.5 MB


In [16]:
CATEGORICAL_FEATURES = [
    'Country'
]

In [17]:
df_test = X_test.copy()
df_test["target"] = y_test

df_train = X_train.copy()
df_train["target"] = y_train

df_val = X_val.copy()
df_val["target"] = y_val

In [18]:
df_val.head()

,Country,StockCode,Year,Month,Day,Hour,min,sec,is_weekend,day_of_week,hour_sin,hour_cos,quarter,stockcode_freq,country_freq,target
255103,United Kingdom,0.389127,2010,6,30,8,14,0,0,2,0.866025,-0.5,2,44,981330,0
255104,United Kingdom,0.080928,2010,6,30,8,14,0,0,2,0.866025,-0.5,2,40,981330,0
255105,United Kingdom,0.038954,2010,6,30,8,14,0,0,2,0.866025,-0.5,2,40,981330,0
255106,United Kingdom,0.132962,2010,6,30,8,14,0,0,2,0.866025,-0.5,2,53,981330,0
255107,United Kingdom,0.427081,2010,6,30,8,14,0,0,2,0.866025,-0.5,2,46,981330,0


In [19]:
known_countries = set(df_train["Country"].unique())

def map_country(x):
    return x if x in known_countries else "UNKNOWN"

df["Country"] = df["Country"].apply(map_country)


In [20]:
df.to_csv('../data/processed/data.csv')

In [21]:
df.head()

,InvoiceDate,Country,StockCode,Year,Month,Day,Hour,min,sec,is_weekend,day_of_week,hour_sin,hour_cos,quarter,stockcode_freq,country_freq,target
0,2009-12-01 07:45:00,United Kingdom,0.466434,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,131,981330,1
1,2009-12-01 07:45:00,United Kingdom,0.551061,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,70,981330,1
2,2009-12-01 07:45:00,United Kingdom,0.547694,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,90,981330,1
3,2009-12-01 07:45:00,United Kingdom,0.385078,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,110,981330,1
4,2009-12-01 07:45:00,United Kingdom,0.210763,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,517,981330,1


In [22]:
df_test.to_csv("../data/processed/test.csv")
df_val.to_csv("../data/processed/val.csv")
df_train.to_csv("../data/processed/train.csv")

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255103 entries, 0 to 255102
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Country         255103 non-null  category
 1   StockCode       255103 non-null  float64 
 2   Year            255103 non-null  int64   
 3   Month           255103 non-null  int64   
 4   Day             255103 non-null  int64   
 5   Hour            255103 non-null  int64   
 6   min             255103 non-null  int64   
 7   sec             255103 non-null  int64   
 8   is_weekend      255103 non-null  int64   
 9   day_of_week     255103 non-null  int64   
 10  hour_sin        255103 non-null  float64 
 11  hour_cos        255103 non-null  float64 
 12  quarter         255103 non-null  int64   
 13  stockcode_freq  255103 non-null  int64   
 14  country_freq    255103 non-null  int64   
dtypes: category(1), float64(3), int64(11)
memory usage: 29.4 MB


In [24]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255103 entries, 0 to 255102
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Country         255103 non-null  category
 1   StockCode       255103 non-null  float64 
 2   Year            255103 non-null  int64   
 3   Month           255103 non-null  int64   
 4   Day             255103 non-null  int64   
 5   Hour            255103 non-null  int64   
 6   min             255103 non-null  int64   
 7   sec             255103 non-null  int64   
 8   is_weekend      255103 non-null  int64   
 9   day_of_week     255103 non-null  int64   
 10  hour_sin        255103 non-null  float64 
 11  hour_cos        255103 non-null  float64 
 12  quarter         255103 non-null  int64   
 13  stockcode_freq  255103 non-null  int64   
 14  country_freq    255103 non-null  int64   
dtypes: category(1), float64(3), int64(11)
memory usage: 29.4 MB


In [26]:
num_pos = np.sum(y_train == 1)
num_neg = np.sum(y_train == 0)
scale_pos_weight = num_neg / num_pos

In [29]:
X_train.head()

,Country,StockCode,Year,Month,Day,Hour,min,sec,is_weekend,day_of_week,hour_sin,hour_cos,quarter,stockcode_freq,country_freq
0,United Kingdom,0.466434,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,131,981330
1,United Kingdom,0.551061,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,70,981330
2,United Kingdom,0.547694,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,90,981330
3,United Kingdom,0.385078,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,110,981330
4,United Kingdom,0.210763,2009,12,1,7,45,0,0,1,0.965926,-0.258819,4,517,981330


In [27]:
# 1. Create Datasets ONCE (Outside the objective function)

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=CATEGORICAL_FEATURES)
dvalid = lgb.Dataset(X_val, label=y_val, categorical_feature=CATEGORICAL_FEATURES, reference=dtrain)


def objective(trial):
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        'scale_pos_weight': scale_pos_weight,
        "feature_pre_filter": False,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True), # Added this!
        "device": "gpu"
    }

    # 2. Add Early Stopping to save time and prevent overfitting
    gbm = lgb.train(
        param, 
        dtrain, 
        valid_sets=[dvalid],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0) # Keeps the console clean
        ]
    )
    
    preds = gbm.predict(X_val)
    pred_labels = (preds >= 0.5).astype(int)
    accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
    return accuracy

In [28]:
study = optuna.create_study(direction="maximize")
study.optimize(objective , n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2026-02-11 15:19:12,056] A new study created in memory with name: no-name-7b2ce125-41bc-4883-97b0-2c7149327142


Training until validation scores don't improve for 50 rounds


[I 2026-02-11 15:19:14,773] Trial 0 finished with value: 0.7797866853332351 and parameters: {'lambda_l1': 0.006967890738524779, 'lambda_l2': 4.5170879029388785e-05, 'num_leaves': 63, 'feature_fraction': 0.6483152056471426, 'bagging_fraction': 0.8049020173313868, 'bagging_freq': 5, 'min_child_samples': 68, 'learning_rate': 0.04247228301824658}. Best is trial 0 with value: 0.7797866853332351.


Early stopping, best iteration is:
[12]	valid_0's binary_logloss: 0.495338
Training until validation scores don't improve for 50 rounds


[I 2026-02-11 15:19:17,249] Trial 1 finished with value: 0.771874884869027 and parameters: {'lambda_l1': 2.3710043250836372e-05, 'lambda_l2': 7.6150988593650295, 'num_leaves': 80, 'feature_fraction': 0.9439210199213165, 'bagging_fraction': 0.9365556493025591, 'bagging_freq': 5, 'min_child_samples': 36, 'learning_rate': 0.05659400656202509}. Best is trial 0 with value: 0.7797866853332351.


...

Early stopping, best iteration is:
[8]	valid_0's binary_logloss: 0.489174
Training until validation scores don't improve for 50 rounds


[I 2026-02-11 15:22:11,300] Trial 8 finished with value: 0.7913550454997605 and parameters: {'lambda_l1': 0.0001082666820297953, 'lambda_l2': 0.0007685694060552914, 'num_leaves': 191, 'feature_fraction': 0.6534721419377091, 'bagging_fraction': 0.6756054332026412, 'bagging_freq': 5, 'min_child_samples': 9, 'learning_rate': 0.09832338264246594}. Best is trial 6 with value: 0.8011365729654054.


Early stopping, best iteration is:
[5]	valid_0's binary_logloss: 0.489922
Training until validation scores don't improve for 50 rounds


[I 2026-02-11 15:22:12,359] Trial 9 finished with value: 0.771874884869027 and parameters: {'lambda_l1': 0.023209808812715525, 'lambda_l2': 4.663342918909319e-07, 'num_leaves': 7, 'feature_fraction': 0.6376403317732354, 'bagging_fraction': 0.715809301303822, 'bagging_freq': 1, 'min_child_samples': 99, 'learning_rate': 0.008665358467112689}. Best is trial 6 with value: 0.8011365729654054.


Early stopping, best iteration is:
[62]	valid_0's binary_logloss: 0.506499
Number of finished trials: 10
Best trial:
  Value: 0.8011365729654054
  Params: 
    lambda_l1: 0.012063082001753914
    lambda_l2: 6.3584879785601
    num_leaves: 172
    feature_fraction: 0.9846822761963748
    bagging_fraction: 0.5678235648463441
    bagging_freq: 3
    min_child_samples: 75
    learning_rate: 0.05916387230263195


In [27]:
from pathlib import Path

Path("../models").mkdir(parents=True, exist_ok=True)



In [28]:
best_params = study.best_params
best_params.update({
    "objective": "binary",
    "metric": "binary_logloss",
    "verbose": -1,
    "boosting_type": "gbdt",
    "device": "gpu",
})

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=CATEGORICAL_FEATURES)
gbm = lgb.train(best_params, dtrain)

gbm.save_model("../models/lgbm_optuna.txt")
